In [1]:
import numpy as np
np.set_printoptions(precision=3, suppress=True)
import pandas as pd
import matplotlib.pyplot as plt
# import numpy.linalg as LA
# import tensorflow as tf
# import torch

from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.insert(0,'/content/drive/My Drive/Bot!')
%cd drive/My\ Drive/Bot!/

Mounted at /content/drive
/content/drive/My Drive/Bot!


In [2]:
# path = 'MarchtoMay_TweetsandNews_fortopicmodeling.csv'
path = 'Processed.csv'
data = pd.read_csv(path)
df = data[['Content','SourceatBot7']]
df = df[df['SourceatBot7'].str.contains("Human") | df['SourceatBot7'].str.contains("Bot")]
df

,Content,SourceatBot7
0,rt dax_x98 ladi gaga 35m covid-19 relief beyon...,Human
1,face covid-19 success medit natur 's real heal...,Bot
2,rt murkymoodr idiot would expect shop tj maxx ...,Bot
3,rt nicolasturgeon stay home protect nhs save live,Human
4,rt dax_x98 ladi gaga 35m covid-19 relief beyon...,Human
...,...,...
119995,rt aspertierra correct https //www.thenation.c...,Human
119996,rt stacy_redvirgo reason doubt number death re...,Bot
119997,rt meganmolteni jetjocko import peopl note mod...,Human
119998,nonprofit empow african american communiti cov...,Bot


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(df['Content'])
vector = vectorizer.transform(df['Content'])

# debug
print(vector.shape)
print(type(vector))
mydict = vectorizer.vocabulary_
print(len(mydict))
print(dict(list(mydict.items())[0:2]))

(117940, 152426)
<class 'scipy.sparse.csr.csr_matrix'>
152426
{'rt': 121578, 'dax_x98': 52386}


In [4]:
X = vector
# X = np.asarray(vector)
y = 1*(df['SourceatBot7'] == 'Bot').to_numpy() # Bot = 1, Human = 0
# print(X[0])
# print(X.shape)
print(y)
print(y.shape)
print(np.count_nonzero(y == 0))
print(np.count_nonzero(y == 1))
print('ratio of human:', np.count_nonzero(y == 0)/y.shape[0])
print('ratio of bot:', np.count_nonzero(y == 1)/y.shape[0])

[0 1 1 ... 0 1 0]
(117940,)
91229
26711
ratio of human: 0.7735204341190436
ratio of bot: 0.2264795658809564


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)


In [6]:
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
import sklearn.metrics as metrics

def classifier(clf):
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print('='*80)
  print(clf)
  print('='*80)
  acc = accuracy_score(y_pred, y_test)
  print('Acc:', format(acc,".4f"))
  print('CM:\n', confusion_matrix(y_test, y_pred))
  tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
  recall = tp / (tp+fn)
  precision = tp / (tp+fp)
  print('recall:\n',recall)
  print("precision:\n", precision)
  f1 = 2*(precision*recall)/(precision+recall)
  print("f1:\n", f1)
  print("f1):\n", f1_score(y_test, y_pred))
  # fpr, tpr, threshold = metrics.roc_curve(y_test, y_pred)
  # roc_auc = metrics.roc_auc_score(y_test, y_pred)
  # print('Area under curve (AUC) = ',roc_auc)
  # plt.xlabel('False Positive Rate')
  # plt.ylabel('True Positive Rate')
  # plt.title('ROC Curve')
  # plt.plot(fpr,tpr)
  acc_list.append(acc)
  f1_list.append(f1)
  return acc

In [7]:
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.linear_model import RidgeClassifier, Perceptron, PassiveAggressiveClassifier, SGDClassifier, LogisticRegression
from sklearn.svm import LinearSVC

acc_list = []
f1_list = []

classifier(LogisticRegression())
classifier(BernoulliNB())
# classifier(MultinomialNB())
classifier(LinearSVC())

# classifier(MultinomialNB(alpha=.01))
# classifier(KNeighborsClassifier(n_neighbors=2))
# classifier(NearestCentroid())
# classifier(RidgeClassifier())
# classifier(Perceptron())
# classifier(PassiveAggressiveClassifier())
# classifier(SGDClassifier())
print(acc_list)
print(f1_list)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()
Acc: 0.7831
CM:
 [[17660   572]
 [ 4544   812]]
recall:
 0.15160567587752055
precision:
 0.5867052023121387
f1:
 0.24094955489614245
f1):
 0.24094955489614245
BernoulliNB()
Acc: 0.7547
CM:
 [[16189  2043]
 [ 3742  1614]]
recall:
 0.30134428678117997
precision:
 0.4413453650533224
f1:
 0.3581493398424498
f1):
 0.3581493398424498
LinearSVC()
Acc: 0.7813
CM:
 [[17031  1201]
 [ 3957  1399]]
recall:
 0.2612023898431665
precision:
 0.5380769230769231
f1:
 0.35168426344896925
f1):
 0.35168426344896925
[0.783110055960658, 0.7547481770391724, 0.7813294895709683]
[0.24094955489614245, 0.3581493398424498, 0.35168426344896925]
